## Import packages

In [68]:
import requests
import pandas as pd

# Reading properties
from jproperties import Properties

# To get Bond rates
import yfinance as yf

## Constants

In [69]:
BASE_URL = 'https://www.alphavantage.co/query?'

## Load properties

In [70]:
# Initial;ize from property file
configs = Properties()

with open('config/av_wacc.properties', 'rb') as config_file:
     configs.load(config_file)

TICKER = configs.get('TICKER').data
# Market return is read as a string; we need to convert it to float
MKT_RETURN = float(configs.get('MKT_RETURN').data)
BOND_TICKER = configs.get('BOND_TICKER').data
API_KEY = configs.get('ALPHAV_API_KEY').data

## Utility method to get data from AV

In [71]:
def get_response(function):
    return requests.get(f'{BASE_URL}function={function}&symbol={TICKER}&apikey={API_KEY}')

## Get income statements from Alpha Vantage

In [72]:
# Income response
latest_income_stmt = get_response('INCOME_STATEMENT').json()['annualReports'][0]

# Balance sheet response
latest_balance_sheet = get_response('BALANCE_SHEET').json()['annualReports'][0]

# Overview for Beta
overview = get_response('OVERVIEW').json()

## Cost of Debt Calculation

In [73]:
# Interest Expense
interest_expense = pd.to_numeric(latest_income_stmt['interestExpense'])

# Income Tax Expense
income_tax = pd.to_numeric(latest_income_stmt['incomeTaxExpense'])

# Income before tax
income_before_tax = pd.to_numeric(latest_income_stmt['incomeBeforeTax'])

# Current debt
current_debt = pd.to_numeric(latest_balance_sheet['shortTermDebt'])

# Long term debt
long_term_debt = pd.to_numeric(latest_balance_sheet['longTermDebtNoncurrent'])

# Cost of debt
cost_of_debt = interest_expense / (current_debt + long_term_debt)

# Effective tax rate
effective_tax_rate = income_tax / income_before_tax

# Cost of debt after tax
cost_of_debt_after_tax = cost_of_debt * (1 - effective_tax_rate)

## Cost of Equity Calculation

In [74]:
# US Treasury Yield 10 years
tnx = yf.Ticker(BOND_TICKER)
# Yield rate
risk_free_rate = tnx.info['regularMarketDayHigh']/100

# Beta
beta = pd.to_numeric(overview['Beta'])

# Cost of equity
cost_of_equity = risk_free_rate + (beta * (MKT_RETURN - risk_free_rate))

## Weight of Debt and Equity Calculation

In [75]:
total_debt = current_debt + long_term_debt

market_cap = pd.to_numeric(overview['MarketCapitalization'])
total = total_debt + market_cap

# Calculate the weight of debt and equity
weight_of_debt = total_debt / total
weight_of_equity = market_cap / total

## WACC Calculation

In [93]:
wacc = (weight_of_debt * cost_of_debt_after_tax) + (weight_of_equity * cost_of_equity)
'{:.2%}'.format(round(wacc,4)) + ' for ticker: ' + TICKER

'7.99% for ticker: MSFT'